# Modelos Epidemiológicos COVID-19

## Introdução
Aqui vamos avaliar diferentes modelos epidemiológicos para tentarmos entender os possíveis caminhos que a COVID-19 pode tomar no Brasil. Apesar de alguns sistemas de equações diferenciais usados aqui terem solução analítica, usaremos sempre um resolvedor numérico.

## Crescimento exponencial
No começo de uma epidemia, é comum vermos crescimento exponencial do número de casos. Por exemplo, aqui está a progressão de casos de COVID-19 no Brasil entre 26/02/2020 e 28/03/2020:

| Data | Casos | Data | Casos | Data | Casos |
|-|-:|-|-:|-|-:|
| 02/26/20 | 1 | 02/27/20 | 1 | 02/28/20 | 1 |
| 02/29/20 | 1 | 03/01/20 | 2 | 03/02/20 | 2 |
| 03/05/20 | 3 | 03/06/20 | 8 | 03/07/20 | 13 |
| 03/08/20 | 13 | 03/09/20 | 25 | 03/11/20 | 34 |
| 03/12/20 | 52 | 03/13/20 | 77 | 03/14/20 | 98 |
| 03/15/20 | 121 | 03/16/20 | 200 | 03/17/20 | 234 |
| 03/18/20 | 291 | 03/19/20 | 428 | 03/20/20 | 621 |
| 03/21/20 | 904 | 03/22/20 | 1128 | 03/23/20 | 1546 |
| 03/24/20 | 1891 | 03/25/20 | 2201 | 03/26/20 | 2433 |
| 03/27/20 | 2915 | 03/28/20 | 3417 
<caption>Fonte: https://ourworldindata.org/coronavirus</caption>

Usando a variável $I$ (e $I_d$ para o número de infectados num determinado dia) para denotar o número de infectados, podemos ver que o crescimento médio é de algo entre 30% e 40% ao dia. Podemos modelar esse comportamento de forma _discreta_ (no sentido matemático) estimando que

$$I_{d+1} = (1+\beta) I_d$$,

onde $\beta$ é a taxa de crescimento, que descobriremos usando aproximações numéricas. Podemos também modelar esse crescimento de maneira _contínua_ usando a equação diferencial

$$I'(t) = \beta I,$$

Onde o símbolo $'$ significa a _derivada_ de $I$. Nesse caso, os dois modelos têm a mesma solução analítica:

$$I(t) = \alpha(1+\beta)^t$$

No gráfico abaixo, podemos ver a comparação entre esse modelo e os dez dias seguintes ao país chegar a 100 casos, já usando a função `curve_fit` para encontrar os valores de $\alpha$ e $\beta$ que melhor aproximam a curva dos dados:

In [11]:
from ipywidgets import interactive, fixed
import numpy as np
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 9]

dados = [121, 200, 234, 291, 428, 621, 904, 1128, 1546, 1891, 2201]

t = np.linspace(0, len(dados)-1, len(dados))
α, b = curve_fit(lambda x, a, b: a*np.exp(b*t),  t,  dados,  p0=(1, 0))[0]

def plt_exp(α=α, β=np.exp(b)-1):
    modelo = [α*(1+β)**x for x in t]

    fig,ax = plt.subplots()
    dados_plot = ax.plot(t, dados, label='Dados')
    modelo_plot = ax.plot(t, modelo, label=f'Modelo, β={β:.3}, α={α:.4}')
    ax.set_xlabel('Dias desde a centésima infecção')
    ax.set_ylabel('Número de infectados')
    ax.set_title('Dados vs Modelo')
    l = ax.legend()
e = interactive(plt_exp, α=(α/2 , 3*α/2, 1), β=(.2, .5, .01))
e

interactive(children=(FloatSlider(value=301859.44018798595, description='α', max=452789.1602819789, min=150929…

# Modelo SI

O problema mais gritante com um modelo exponencial é que uma epidemia obviamente não pode crescer indefinidamente, já que o número de pessoas suscetíveis é limitado. Não só isso, conforme mais pessoas são infectadas, o número de novas infecções diminuirá, já que não acontece uma nova infecção numa interação entre duas pessoas já infectadas.

<p align="center">
  <img src="si.png">
</p>

Assim, temos que a taxa de novas infecções é dada por $\beta$, multiplicado pela fração de pessoas ainda suscetíveis, $S/N$, onde $N$ é o total da população:
$$
\begin{align}
  S'(t) & = -\beta I\frac{S}{N}\\
  I'(t) & = \beta I\frac{S}{N}\\
\end{align}
$$

In [2]:
from scipy import integrate

def solve_sir(S_i=210_000_000., I_i=2200., β=0.16, dias=180, N=24):
    """Plot a solution to the SI differential equations."""
    
    def sir_deriv(s_i, t0, β=β):
        """Compute the time-derivative of a SI system."""
        s, i = s_i
        n = s + i
        return [
            -β * i * s/n, 
            β * i * s/n, 
        ]
    
    t = np.linspace(0,dias,int(N*dias))
    s_i_t = integrate.odeint(sir_deriv, (S_i, I_i), t)
    fig,ax = plt.subplots()
    ax.stackplot(t, s_i_t[:,1], s_i_t[:,0],
                 labels=('Infecciosos', 'Suscetíveis'))
    ax.set_xlabel('Dias')
    ax.set_ylabel('Pessoas')
    ax.set_title(f'Modelo SI')
    l = ax.legend()
    return t, s_i_t
w=interactive(solve_sir, S_i=fixed(210_000_000), I_i=fixed(2200), β=(0.0, 0.5, 0.01), N=fixed(10))
w

interactive(children=(FloatSlider(value=0.16, description='β', max=0.5, step=0.01), IntSlider(value=180, descr…

## Modelo SIR
O modelo mais usado para modelar epidemias é o SIR. Nele, presume-se que as pessoas podem passar por três estágios: Suscetível, Infeccioso e Resolvido. Por enquanto, esse modelo parece apropriado para essa epidemia, já que a hipótese de uma reinfecção parece remota.

<p align="center">
  <img src="sir.png">
</p>

### Premissas do modelo SIR

Vamos usar as letras $S$ para suscetível, $I$ para infeccioso e $R$ para Resolvido e $N$ para a população inicial total, ou seja, a soma $S+I+R$. O modelo presume que cada população começa com um determinado valor e tenta prever a evolução futura delas. Para isso, ele define as taxas de variação de cada uma das funções. O número de suscetíveis que se torna infectado é dado por $\beta I\frac{S}{N}$, onde $\beta$ é quantos novos casos por dia são causados por cada infectado, em média; $S/N$ é a fração de suscetíveis sobre a população total. Já a mudança nas resoluções (curas ou mortes) é dado por $I/\mu$, onde $\mu$ é a média de dias até a resolução. Assim, temos:

$$
\begin{align}
  S'(t) & = -\beta I\frac{S}{N}\\
  I'(t) & = \beta I\frac{S}{N} - \frac{I}{\mu}\\
  R'(t) & = \frac{I}{\mu}
\end{align}
$$

Uma medida que foi bastante falada na mídia desde o início dessa pandemia é o _número básico de reprodução_, normalmente denotado por $R_0$ (sem relação com o $R$ acima). Nesse modelo, o número é dado por $\beta\mu$, ou seja, o número médio de novas infecções a cada dia por infectado, multiplicado pelo número médio de dias de infecção.

In [3]:
def solve_sir(S_i=210_000_000., I_i=2200., R_i=100., β=0.16, μ=14, dias=365, N=24):
    """Plot a solution to the SIR differential equations."""
    
    def sir_deriv(s_i_r, t0, β=β, μ=μ):
        """Compute the time-derivative of a SIR system."""
        s, i, r = s_i_r
        n = s + i + r
        return [
            -β * i * s/n, 
            β * i * s/n - i/μ, 
            i/μ
        ]
    
    t = np.linspace(0,dias,int(N*dias))
    s_i_r_t = integrate.odeint(sir_deriv, (S_i, I_i, R_i), t)
    fig,ax = plt.subplots()
    ax.stackplot(t, s_i_r_t[:,1], s_i_r_t[:,0], s_i_r_t[:,2],
                 labels=('Infecciosos', 'Suscetíveis', 'Resolvidos'))
    ax.set_xlabel('Dias')
    ax.set_ylabel('Pessoas')
    ax.set_title(f'Modelo SIR, R₀={β*μ:.3}')
    l = ax.legend()
    demanda_uti = int(max(s_i_r_t[:,1])*.01)
    print(f'Máximo de casos graves simultâneos: {demanda_uti:,}')
    resolvidos = s_i_r_t[-1][-1]
    mortes_estimadas = int(resolvidos * .005)
    print(f'Mortes estimadas indepedente de UTI: {mortes_estimadas:,}')
    print(f'Pessoas ainda suscetíveis ao final: {int(s_i_r_t[-1][0]):,}')
    return t, s_i_r_t

w=interactive(solve_sir, S_i=fixed(210_000_000), I_i=fixed(2200), R_i=fixed(100), 
              β=(0.0, 0.5, 0.01), mu=(1,30), N=fixed(10))
w

interactive(children=(FloatSlider(value=0.16, description='β', max=0.5, step=0.01), IntSlider(value=14, descri…

## Simulações

#TODO enquanto isso, assista esse vídeo aqui: https://www.youtube.com/watch?v=gxAaO2rsdIs